# Querying hierarchical data with Rabbit
We reproduce the queries from the section on querying hierarcical data using Rabbit syntax.

## The database
We start with loading a sample hierarchical database.  Our sample database is derived from the dataset of all employees of the city of Chicago ([source](https://data.cityofchicago.org/Administration-Finance/Current-Employee-Names-Salaries-and-Position-Title/xzkq-xp2w)).

In [1]:
ENV["LINES"] = 15
push!(LOAD_PATH, "..")
using RBT
include("../citydb.jl")

setdb(citydb)

Department:
  name: UTF8String {exclusive}
  employee: Employee {inverse of employee.department}
Employee:
  surname: UTF8String
  name: UTF8String
  department: Department
  position: UTF8String
  salary: Int64

We can execute a query using `@query()` command:

In [2]:
@query(6*(3+4))

42

## Traversing the hierarchy
*Find the names of all departments.*

In [3]:
@query(department.name)

35-element Array{UTF8String,1}:
 "WATER MGMNT"      
 "POLICE"           
 "GENERAL SERVICES" 
 "CITY COUNCIL"     
 "STREETS & SAN"    
 ⋮                  
 "BOARD OF ETHICS"  
 "POLICE BOARD"     
 "BUDGET & MGMT"    
 "ADMIN HEARNG"     
 "LICENSE APPL COMM"

*Find the names of all employees.*

In [4]:
@query(department.employee.name)

32181-element Array{UTF8String,1}:
 "ELVIA"     
 "VICENTE"   
 "MUHAMMAD"  
 "GIRLEY"    
 "DILAN"     
 ⋮           
 "NANCY"     
 "DARCI"     
 "THADDEUS"  
 "RACHENETTE"
 "MICHELLE"  

We are not restricted by the hierarchical structure of the database, so we can query employees directly.

In [5]:
@query(employee.name)

32181-element Array{UTF8String,1}:
 "ELVIA"    
 "JEFFERY"  
 "KARINA"   
 "KIMBERLEI"
 "VICENTE"  
 ⋮          
 "MICHAEL"  
 "PETER"    
 "MARK"     
 "CARLO"    
 "DARIUSZ"  

If the traversal ends at an entity class, an array of records is generated.

In [6]:
@query(department)

35-element Array{Tuple{UTF8String},1}:
 ("WATER MGMNT",)      
 ("POLICE",)           
 ("GENERAL SERVICES",) 
 ("CITY COUNCIL",)     
 ("STREETS & SAN",)    
 ⋮                     
 ("BOARD OF ETHICS",)  
 ("POLICE BOARD",)     
 ("BUDGET & MGMT",)    
 ("ADMIN HEARNG",)     
 ("LICENSE APPL COMM",)

Which fields are selected depends on the path to the class.

In [7]:
@query(employee)

32181-element Array{Tuple{UTF8String,UTF8String,UTF8String,UTF8String,Int64},1}:
 ("ELVIA","A","WATER MGMNT","WATER RATE TAKER",88968)                     
 ("JEFFERY","A","POLICE","POLICE OFFICER",80778)                          
 ("KARINA","A","POLICE","POLICE OFFICER",80778)                           
 ("KIMBERLEI","A","GENERAL SERVICES","CHIEF CONTRACT EXPEDITER",84780)    
 ("VICENTE","A","WATER MGMNT","CIVIL ENGINEER IV",104736)                 
 ⋮                                                                        
 ("MICHAEL","Z","GENERAL SERVICES","FRM OF MACHINISTS - AUTOMOTIVE",97448)
 ("PETER","Z","POLICE","POLICE OFFICER",86520)                            
 ("MARK","Z","POLICE","POLICE OFFICER",83616)                             
 ("CARLO","Z","POLICE","POLICE OFFICER",86520)                            
 ("DARIUSZ","Z","DoIT","CHIEF DATA BASE ANALYST",110352)                  

In [8]:
@query(department.employee)

32181-element Array{Tuple{UTF8String,UTF8String},1}:
 ("ELVIA","A")     
 ("VICENTE","A")   
 ("MUHAMMAD","A")  
 ("GIRLEY","A")    
 ("DILAN","A")     
 ⋮                 
 ("NANCY","T")     
 ("DARCI","W")     
 ("THADDEUS","W")  
 ("RACHENETTE","W")
 ("MICHELLE","G")  

## Summarizing data
*Find the number of departments.*

In [9]:
@query(count(department))

35

*Find the number of employees for each department.*

In [10]:
@query(department.count(employee))

35-element Array{Int64,1}:
  1848
 13570
   924
   397
  2090
     ⋮
     9
     2
    43
    39
     1

*Find the total number of employees.*

In [11]:
@query(count(department.employee))

32181

Again, we can query `employee` directly.

In [12]:
@query(count(employee))

32181

*Find the top salary among all employees.*

In [13]:
@query(max(employee.salary))

Nullable(260004)

*Find the maximum number of employees per department.*

In [14]:
@query(max(department.count(employee)))

Nullable(13570)

## Tabular output
*For each department, find the number of employees.*

In [15]:
@query(department:select(name,count(employee)))

35-element Array{Tuple{UTF8String,Int64},1}:
 ("WATER MGMNT",1848)    
 ("POLICE",13570)        
 ("GENERAL SERVICES",924)
 ("CITY COUNCIL",397)    
 ("STREETS & SAN",2090)  
 ⋮                       
 ("BOARD OF ETHICS",9)   
 ("POLICE BOARD",2)      
 ("BUDGET & MGMT",43)    
 ("ADMIN HEARNG",39)     
 ("LICENSE APPL COMM",1) 

The `:select` notation is a syntax sugar for regular function call where the first argument is placed before the function name (postfix notation).

In [16]:
@query(select(department,name,count(employee)))

35-element Array{Tuple{UTF8String,Int64},1}:
 ("WATER MGMNT",1848)    
 ("POLICE",13570)        
 ("GENERAL SERVICES",924)
 ("CITY COUNCIL",397)    
 ("STREETS & SAN",2090)  
 ⋮                       
 ("BOARD OF ETHICS",9)   
 ("POLICE BOARD",2)      
 ("BUDGET & MGMT",43)    
 ("ADMIN HEARNG",39)     
 ("LICENSE APPL COMM",1) 

It is easy to add new columns to the output.  Let us add *the top salary per department.*

In [17]:
@query(
    department
    :select(
        name,
        count(employee),
        max(employee.salary)))

35-element Array{Tuple{UTF8String,Int64,Nullable{Int64}},1}:
 ("WATER MGMNT",1848,Nullable(169512))    
 ("POLICE",13570,Nullable(260004))        
 ("GENERAL SERVICES",924,Nullable(157092))
 ("CITY COUNCIL",397,Nullable(160248))    
 ("STREETS & SAN",2090,Nullable(157092))  
 ⋮                                        
 ("BOARD OF ETHICS",9,Nullable(131688))   
 ("POLICE BOARD",2,Nullable(97728))       
 ("BUDGET & MGMT",43,Nullable(169992))    
 ("ADMIN HEARNG",39,Nullable(156420))     
 ("LICENSE APPL COMM",1,Nullable(69888))  

## Filtering data
*Find the employees with salary greater than $200k.*

In [18]:
@query(
    employee
    :filter(salary>200000)
    :select(name,surname,position,salary))

3-element Array{Tuple{UTF8String,UTF8String,UTF8String,Int64},1}:
 ("RAHM","E","MAYOR",216210)                    
 ("GARRY","M","SUPERINTENDENT OF POLICE",260004)
 ("JOSE","S","FIRE COMMISSIONER",202728)        

`:select()` and `:filter()` can be applied in any order.

In [19]:
@query(
    employee
    :select(name,surname,position,salary)
    :filter(salary>200000))

3-element Array{Tuple{UTF8String,UTF8String,UTF8String,Int64},1}:
 ("RAHM","E","MAYOR",216210)                    
 ("GARRY","M","SUPERINTENDENT OF POLICE",260004)
 ("JOSE","S","FIRE COMMISSIONER",202728)        

*Find the number of employees with salary in the range from \$100k to \$200k.*

In [20]:
@query(
    employee
    :filter((salary>100000)&(salary<=200000))
    :count)

3916

*Find the departments with mode than 1000 employees.*

In [21]:
@query(
    department
    :filter(count(employee)>1000)
    .name)


Use "filter(count(employee)>1000)." instead.


7-element Array{UTF8String,1}:
 "WATER MGMNT"  
 "POLICE"       
 "STREETS & SAN"
 "AVIATION"     
 "FIRE"         
 "OEMC"         
 "TRANSPORTN"   

*Find the number of departments with more than 1000 employees.*

In [22]:
@query(
    count(
        department
        :filter(count(employee)>1000)))

7

*For each department, find the number of employees with salary higher than $100k.*

In [23]:
@query(
    department
    :select(
        name,
        count(employee:filter(salary>100000))))

35-element Array{Tuple{UTF8String,Int64},1}:
 ("WATER MGMNT",179)    
 ("POLICE",1493)        
 ("GENERAL SERVICES",79)
 ("CITY COUNCIL",54)    
 ("STREETS & SAN",39)   
 ⋮                      
 ("BOARD OF ETHICS",2)  
 ("POLICE BOARD",0)     
 ("BUDGET & MGMT",12)   
 ("ADMIN HEARNG",3)     
 ("LICENSE APPL COMM",0)

*For each department with the number of employees less than 1000, find the employees with salary higher than $125k.*

In [24]:
@query(
    department
    :filter(count(employee)<1000)
    :select(
        name,
        employee
            :filter(salary>125000)
            :select(name,surname,position)))

28-element Array{Tuple{UTF8String,Array{Tuple{UTF8String,UTF8String,UTF8String},1}},1}:
 ("GENERAL SERVICES",[("DAVID","R","COMMISSIONER OF FLEET & FACILITY MANAGEMENT"),("PHILLIP","S","EQUIPMENT SERVICES COORD")])                                                            
 ("CITY COUNCIL",[("JAMES","C","DEPUTY CHIEF ADMINISTRATIVE OFFICER"),("MARLA","K","CHIEF ADMINISTRATIVE OFFICER")])                                                                      
 ("FAMILY & SUPPORT",[("EVELYN","D","COMMISSIONER OF FAMILY AND SUPPORT SERVICES"),("MARY","G","DEPUTY COMMISSIONER"),("JENNIFER","W","FIRST DEPUTY COMMISSIONER")])                      
 ("IPRA",[("SCOTT","A","CHIEF ADMINISTRATOR"),("STEVEN","H","DEPUTY CHIEF ADMINISTRATOR"),("STEVEN","M","FIRST DEPUTY CHIEF ADMINISTRATOR"),("WILLIAM","W","DEPUTY CHIEF ADMINISTRATOR")])
 ("PUBLIC LIBRARY",[("BRIAN","B","COMMISSIONER OF CHICAGO PUBLIC LIBRARY"),("MICHELLE","F","DIRECTOR OF LIBRARY TECHNOLOGY"),("ANDREA","S","FIRST DEPUTY COMMISSIONE

## Sorting

We use the `:sort` combinator to sort an array of values.

*List the names of departments in alphabetical order.*

In [25]:
@query(department.name:sort)

35-element Array{UTF8String,1}:
 "ADMIN HEARNG"     
 "ANIMAL CONTRL"    
 "AVIATION"         
 "BOARD OF ELECTION"
 "BOARD OF ETHICS"  
 ⋮                  
 "PUBLIC LIBRARY"   
 "STREETS & SAN"    
 "TRANSPORTN"       
 "TREASURER"        
 "WATER MGMNT"      

We can also specify the attribute by which the elements of the array are to be sorted.

*Show the employees sorted by salary.*

In [26]:
@query(employee:sort(salary))

32181-element Array{Tuple{UTF8String,UTF8String,UTF8String,UTF8String,Int64},1}:
 ("STEVEN","K","MAYOR'S OFFICE","ADMINISTRATIVE SECRETARY",1)  
 ("BETTY","A","FAMILY & SUPPORT","FOSTER GRANDPARENT",2756)    
 ("VICTOR","A","FAMILY & SUPPORT","SENIOR COMPANION",2756)     
 ("RASHEEDAH","A","FAMILY & SUPPORT","SENIOR COMPANION",2756)  
 ("MARY","A","FAMILY & SUPPORT","FOSTER GRANDPARENT",2756)     
 ⋮                                                             
 ("CHARLES","S","FIRE","FIRST DEPUTY FIRE COMMISSIONER",197736)
 ("ALFONZA","W","POLICE","FIRST DEPUTY SUPERINTENDENT",197736) 
 ("JOSE","S","FIRE","FIRE COMMISSIONER",202728)                
 ("RAHM","E","MAYOR'S OFFICE","MAYOR",216210)                  
 ("GARRY","M","POLICE","SUPERINTENDENT OF POLICE",260004)      

Use `:desc` indicator to reverse the order.

In [27]:
@query(employee:sort(salary:desc))

32181-element Array{Tuple{UTF8String,UTF8String,UTF8String,UTF8String,Int64},1}:
 ("GARRY","M","POLICE","SUPERINTENDENT OF POLICE",260004)      
 ("RAHM","E","MAYOR'S OFFICE","MAYOR",216210)                  
 ("JOSE","S","FIRE","FIRE COMMISSIONER",202728)                
 ("CHARLES","S","FIRE","FIRST DEPUTY FIRE COMMISSIONER",197736)
 ("ALFONZA","W","POLICE","FIRST DEPUTY SUPERINTENDENT",197736) 
 ⋮                                                             
 ("SONG","Y","FAMILY & SUPPORT","SENIOR COMPANION",2756)       
 ("AMANDA","Y","FAMILY & SUPPORT","FOSTER GRANDPARENT",2756)   
 ("EUNICE","Y","FAMILY & SUPPORT","SENIOR COMPANION",2756)     
 ("MING","Y","FAMILY & SUPPORT","SENIOR COMPANION",2756)       
 ("STEVEN","K","MAYOR'S OFFICE","ADMINISTRATIVE SECRETARY",1)  

It is possible to specify several sorting keys.

In [28]:
@query(
    employee
    :sort(
        salary:desc,
        surname:asc,
        name:asc))

32181-element Array{Tuple{UTF8String,UTF8String,UTF8String,UTF8String,Int64},1}:
 ("GARRY","M","POLICE","SUPERINTENDENT OF POLICE",260004)      
 ("RAHM","E","MAYOR'S OFFICE","MAYOR",216210)                  
 ("JOSE","S","FIRE","FIRE COMMISSIONER",202728)                
 ("CHARLES","S","FIRE","FIRST DEPUTY FIRE COMMISSIONER",197736)
 ("ALFONZA","W","POLICE","FIRST DEPUTY SUPERINTENDENT",197736) 
 ⋮                                                             
 ("MING","Y","FAMILY & SUPPORT","SENIOR COMPANION",2756)       
 ("SONG","Y","FAMILY & SUPPORT","SENIOR COMPANION",2756)       
 ("SONG","Y","FAMILY & SUPPORT","SENIOR COMPANION",2756)       
 ("YELENA","Y","FAMILY & SUPPORT","SENIOR COMPANION",2756)     
 ("STEVEN","K","MAYOR'S OFFICE","ADMINISTRATIVE SECRETARY",1)  

`:sort` can be used together with `:select` and `:filter`.

In [29]:
@query(
    department
    :select(name, count(employee))
    :filter(count(employee)>1000)
    :sort(count(employee):desc))

7-element Array{Tuple{UTF8String,Int64},1}:
 ("POLICE",13570)      
 ("FIRE",4875)         
 ("STREETS & SAN",2090)
 ("WATER MGMNT",1848)  
 ("AVIATION",1344)     
 ("TRANSPORTN",1200)   
 ("OEMC",1135)         

Use `:define` to name a commonly used expression.

In [30]:
@query(
    department
    :define(size => count(employee))
    :select(name, size)
    :filter(size>1000)
    :sort(size:desc))

7-element Array{Tuple{UTF8String,Int64},1}:
 ("POLICE",13570)      
 ("FIRE",4875)         
 ("STREETS & SAN",2090)
 ("WATER MGMNT",1848)  
 ("AVIATION",1344)     
 ("TRANSPORTN",1200)   
 ("OEMC",1135)         

## Grouping
Use `:unique` combinator to generate all unique values that appears in a sequence.

*List all distinct positions.*

In [31]:
@query(employee.position:unique)

1094-element Array{UTF8String,1}:
 "1ST DEPUTY INSPECTOR GENERAL"
 "A/MGR COM SVC-ELECTIONS"     
 "A/MGR OF MIS-ELECTIONS"      
 "A/MGR WAREHOUSE-ELECTIONS"   
 "A/SUPRV REDISTRICTING"       
 ⋮                             
 "WINDOW WASHER"               
 "YOUTH SERVICES COORD"        
 "ZONING ADMINISTRATOR"        
 "ZONING INVESTIGATOR"         
 "ZONING PLAN EXAMINER"        

*Find the number of distinct positions for each department.*

In [32]:
@query(
    department
    :select(
        name,
        count(unique(employee.position)),
        count(employee)))

35-element Array{Tuple{UTF8String,Int64,Int64},1}:
 ("WATER MGMNT",154,1848)    
 ("POLICE",129,13570)        
 ("GENERAL SERVICES",119,924)
 ("CITY COUNCIL",28,397)     
 ("STREETS & SAN",70,2090)   
 ⋮                           
 ("BOARD OF ETHICS",9,9)     
 ("POLICE BOARD",2,2)        
 ("BUDGET & MGMT",24,43)     
 ("ADMIN HEARNG",15,39)      
 ("LICENSE APPL COMM",1,1)   

We can also list distinct positions using `:by` combinator.

In [33]:
@query(employee:by(position))

1094-element Array{Tuple{UTF8String},1}:
 ("1ST DEPUTY INSPECTOR GENERAL",)
 ("A/MGR COM SVC-ELECTIONS",)     
 ("A/MGR OF MIS-ELECTIONS",)      
 ("A/MGR WAREHOUSE-ELECTIONS",)   
 ("A/SUPRV REDISTRICTING",)       
 ⋮                                
 ("WINDOW WASHER",)               
 ("YOUTH SERVICES COORD",)        
 ("ZONING ADMINISTRATOR",)        
 ("ZONING INVESTIGATOR",)         
 ("ZONING PLAN EXAMINER",)        

For each row generated by `employee:by(position)` combinator `employee` will give you employees that have this position.

*For each position, find the number of employees.*

In [34]:
@query(
    employee
    :by(position)
    :select(position, count(employee))
    :sort(count(employee):desc))

1094-element Array{Tuple{UTF8String,Int64},1}:
 ("POLICE OFFICER",9489)                       
 ("FIREFIGHTER-EMT",1191)                      
 ("SERGEANT",1138)                             
 ("FIREFIGHTER",970)                           
 ("POLICE OFFICER (ASSIGNED AS DETECTIVE)",808)
 ⋮                                             
 ("WEB AUTHOR",1)                              
 ("WEB DEVELOPER",1)                           
 ("WEB DEVELOPER/ADMINTR-CPL",1)               
 ("ZONING ADMINISTRATOR",1)                    
 ("ZONING INVESTIGATOR",1)                     

*Find positions provided by no less than 5 departments.*

In [35]:
@query(
    employee:by(position)
    :define(department => unique(employee.department))
    :filter(count(department)>=5)
    :select(position, department)
    :sort(count(department):desc))

78-element Array{Tuple{UTF8String,Array{UTF8String,1}},1}:
 ("STAFF ASST",UTF8String["WATER MGMNT","POLICE","GENERAL SERVICES","STREETS & SAN","AVIATION","FIRE","FAMILY & SUPPORT","PUBLIC LIBRARY","DoIT","BUSINESS AFFAIRS"  …  "BUILDINGS","ANIMAL CONTRL","CITY CLERK","INSPECTOR GEN","TREASURER","DISABILITIES","PROCUREMENT","BOARD OF ETHICS","ADMIN HEARNG","LICENSE APPL COMM"])                    
 ("ADMINISTRATIVE SERVICES OFFICER II",UTF8String["WATER MGMNT","POLICE","STREETS & SAN","AVIATION","FAMILY & SUPPORT","PUBLIC LIBRARY","DoIT","BUSINESS AFFAIRS","OEMC","TRANSPORTN"  …  "LAW","COMMUNITY DEVELOPMENT","BUILDINGS","ANIMAL CONTRL","CITY CLERK","DISABILITIES","HUMAN RESOURCES","HUMAN RELATIONS","BUDGET & MGMT","ADMIN HEARNG"])
 ("ADMINISTRATIVE ASST II",UTF8String["WATER MGMNT","POLICE","GENERAL SERVICES","STREETS & SAN","AVIATION","FIRE","FAMILY & SUPPORT","PUBLIC LIBRARY","BUSINESS AFFAIRS","OEMC","TRANSPORTN","HEALTH","LAW","FINANCE","BUILDINGS","CITY CLERK","PROCUREMENT","ADMIN

*Find the popular names of Chicago employees.*

In [36]:
@query(
    employee:by(name)
    :select(name, count(employee))
    :sort(count(employee):desc))

5038-element Array{Tuple{UTF8String,Int64},1}:
 ("MICHAEL",1170)
 ("JOHN",914)    
 ("JAMES",691)   
 ("ROBERT",649)  
 ("JOSEPH",543)  
 ⋮               
 ("ZORAN",1)     
 ("ZORRAYDA",1)  
 ("ZULEMA",1)    
 ("ZUZANNA",1)   
 ("ZYNETTA",1)   

*Find the top salary by the first name, but only if there are at least 10 employees having this name.*

In [37]:
@query(
    employee:by(name)
    :filter(count(employee)>=10)
    :select(name, max(employee.salary))
    :sort(max(employee.salary):desc))

483-element Array{Tuple{UTF8String,Int64},1}:
 ("JOSE",202728)   
 ("CHARLES",197736)
 ("ROBERT",194256) 
 ("ANTHONY",187680)
 ("JOHN",187680)   
 ⋮                 
 ("SARA",86520)    
 ("TERRI",86520)   
 ("DARIUS",85020)  
 ("ANNIE",83616)   
 ("BERTHA",83616)  

## Compiling and executing queries
You can compile and execute queries separately.  To compile a query, use the `RBT.compile()` function.

In [38]:
q1 = RBT.@prepare(department.employee.name)

department.employee.name :: Array{UTF8String,1}

In [39]:
q2 = RBT.@prepare(count(department))

count(department) :: Int64

In [40]:
q3 = RBT.@prepare(department:select(name,count(employee)))

select(department,name,count(employee)) :: Array{Tuple{UTF8String,Int64},1}

In [41]:
q4 = RBT.@prepare(count(employee:filter((salary>100000)&(salary<200000))))

count(filter(employee,&(>(salary,100000),<(salary,200000)))) :: Int64

To execute a query, call the compiled query as a function.

In [42]:
q1()

32181-element Array{UTF8String,1}:
 "ELVIA"     
 "VICENTE"   
 "MUHAMMAD"  
 "GIRLEY"    
 "DILAN"     
 ⋮           
 "NANCY"     
 "DARCI"     
 "THADDEUS"  
 "RACHENETTE"
 "MICHELLE"  

In [43]:
q2()

35

In [44]:
q3()

35-element Array{Tuple{UTF8String,Int64},1}:
 ("WATER MGMNT",1848)    
 ("POLICE",13570)        
 ("GENERAL SERVICES",924)
 ("CITY COUNCIL",397)    
 ("STREETS & SAN",2090)  
 ⋮                       
 ("BOARD OF ETHICS",9)   
 ("POLICE BOARD",2)      
 ("BUDGET & MGMT",43)    
 ("ADMIN HEARNG",39)     
 ("LICENSE APPL COMM",1) 

In [45]:
q4()

3916